# Denison DA210/CS181 Homework 4.b - Step 2

Before you turn this notebook in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells**.

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds(source="sqlite1")
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/book.db


#### Establish Connection from Client to Server

In [4]:
%sql $cstring

---

## Part B: SQL Group By Operations

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises._

**Q5:** Using the SQL table `countries` in the `book` database, use a select query to answer the question: how many countries are there in each region?  Select the `region` field as well as a new column, `new`, which answers this question.

In [37]:
query5 = """
SELECT region, COUNT( country) AS new
FROM countries
GROUP BY region
"""
resultset5 = %sql $query5
resultdf5 = resultset5.DataFrame()
resultdf5

 * sqlite:///../../dbfiles/book.db
Done.


,region,new
0,East Asia & Pacific,37
1,Europe & Central Asia,58
2,Latin America & Caribbean,42
3,Middle East & North Africa,21
4,North America,3
5,South Asia,8
6,Sub-Saharan Africa,48


In [38]:
# Testing cell
assert len(resultdf5) == 7
assert len(resultdf5.columns) == 2
assert 58 in list(resultdf5['new'])
assert resultdf5.set_index("region").loc["North America", "new"] == 3

**Q6:** Use the `indicators` table in the `book` database to find the total world population in each year (as the sum of country populations). Use the alias `total_pop` for your new column, and select the `year` and `total_pop` fields.  Sort the result in descending `year` value.

In [39]:
query6 = """
SELECT year, TOTAL(pop) as total_pop
FROM indicators
GROUP BY year
ORDER BY year DESC 
"""
resultset6 = %sql $query6
resultdf6 = resultset6.DataFrame()
resultdf6.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,total_pop
0,2018,7567.31
1,2017,7484.03
2,2016,7399.25
3,2015,7313.74
4,2014,7228.98


In [40]:
# Testing cell
assert len(resultdf6) == 59
assert resultdf6.loc[0,'total_pop'] > 7500
assert resultdf6.loc[0,'total_pop'] < 7600
assert resultdf6.loc[58,'total_pop'] > 3014
assert resultdf6.loc[58,'total_pop'] < 3015

**Q7:** Treating your query above as a subquery (without the ORDER BY), find the minimum for `total_pop` over all years. Use the alias `m` for the new column.

In [44]:
query7 = """
SELECT MIN(total_pop) AS m
FROM (SELECT year, TOTAL(pop) as total_pop
FROM indicators
GROUP BY year
ORDER BY year DESC)
"""

resultset7 = %sql $query7
resultdf7 = resultset7.DataFrame()
resultdf7.head()

 * sqlite:///../../dbfiles/book.db
Done.


,m
0,3014.69


In [45]:
# Testing cell
assert len(resultdf7) == 1
assert resultdf7.loc[0,'m'] > 3014
assert resultdf7.loc[0,'m'] < 3015

**Q8:** Not all countries are growing, so the largest population a country ever had might be in a previous year. For each country code in `indicators`, find the max population that country ever had. Alias your new column as `max_pop`. You should have one row per country, and select the fields `code` and `max_pop`. Don't change the order (that is, your records should still be ordered by `code`).

In [46]:
query8 = """
SELECT code, MAX(pop) AS max_pop
FROM indicators
GROUP BY code
"""

resultset8 = %sql $query8
resultdf8 = resultset8.DataFrame()
resultdf8.head()

 * sqlite:///../../dbfiles/book.db
Done.


,code,max_pop
0,ABW,0.11
1,AFG,37.17
2,AGO,30.81
3,ALB,3.29
4,AND,0.08


In [47]:
# Testing cell
assert len(resultdf8) == 218
assert resultdf8.loc[0,'max_pop'] == 0.11
assert resultdf8.loc[1,'max_pop'] == 37.17

**Q9:** Copy your query from the previous question, and then modify it to find all records where the max population is less than 1 (remember, this is measured in millions of people).  Use a `HAVING` clause.  Keep the original ordering of the data (alphabetically, by `code`).

In [50]:
query9 = """
SELECT code, MAX(pop) AS max_pop
FROM indicators
GROUP BY code
HAVING max_pop < 1
"""

resultset9 = %sql $query9
resultdf9 = resultset9.DataFrame()
resultdf9.head()

 * sqlite:///../../dbfiles/book.db
Done.


,code,max_pop
0,ABW,0.11
1,AND,0.08
2,ASM,0.06
3,ATG,0.10
4,BHS,0.39


In [51]:
# Testing cell
assert len(resultdf9) == 58
assert resultdf9.loc[0,'max_pop'] == 0.11
assert resultdf9.loc[1,'max_pop'] == 0.08

**Q10:** Using the `indicators` table in the `book` database, find the total world population in each year (as the sum of country populations), then return the rows where the total population is greater than 6000 (measured in millions of people).  Use the alias `total_pop` for your new column.

In [53]:
query10 = """
SELECT SUM(pop) AS total_pop 
FROM indicators 
GROUP BY year 
HAVING total_pop > 6000
"""

resultset10 = %sql $query10
resultdf10 = resultset10.DataFrame()
resultdf10.head()

 * sqlite:///../../dbfiles/book.db
Done.


,total_pop
0,6013.26
1,6092.92
2,6172.14
3,6251.05
4,6330.08


In [54]:
# Testing cell
assert len(resultdf10) == 20
assert resultdf10.loc[0,'total_pop'] < 6014
assert resultdf10.loc[1,'total_pop'] > 6090

---

---

## Part C

**Q11:** How much time (in minutes/hours) did you spend on this homework assignment?

15 mins 

**Q12:** Who was your partner for this assignment?  If you worked alone, say so instead.

Alone

